### Exemplo Classico de Contagem de Palavras

**Setup**

In [2]:
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql.window import Window

In [3]:
#SparkSession
spark = (SparkSession.builder 
        .master("local[*]") 
        .appName("workshop_spark")
        .getOrCreate())

In [4]:
spark

In [5]:
#SparkContext - sc é o nome padrão
sc = spark.sparkContext

In [6]:
#Criando RDD com o arquivo de texto
pathRDD = sc.textFile("../data/shakespeare.txt")
print(pathRDD.collect())

[' “No entanto, para dizer a verdade, hoje em dia a razão e o amor quase não andam juntos. É pena que alguns vizinhos honestos não se esforcem para torná-los amigos. Como vê, também posso ser espirituoso se houver ocasião.” (De Sonho de uma Noite de Verão)', '', ' “A vida é uma simples sombra que passa (...); é uma história contada por um idiota, cheia de ruído e de furor e que nada significa” (De Macbeth)']


In [7]:
#Eliminando as linhas vazias do arquivo
pathRDDFilter = pathRDD.filter(lambda x: len(x)>0)
print(pathRDDFilter.collect())

[' “No entanto, para dizer a verdade, hoje em dia a razão e o amor quase não andam juntos. É pena que alguns vizinhos honestos não se esforcem para torná-los amigos. Como vê, também posso ser espirituoso se houver ocasião.” (De Sonho de uma Noite de Verão)', ' “A vida é uma simples sombra que passa (...); é uma história contada por um idiota, cheia de ruído e de furor e que nada significa” (De Macbeth)']


In [8]:
#Dividindo o conteúdo com base no espaço entre as palavras
#Pegamos apenas as linhas com conteúdo
wordsRDD = pathRDDFilter.flatMap(lambda x: x.split(" "))

In [9]:
print(wordsRDD.collect())

['', '“No', 'entanto,', 'para', 'dizer', 'a', 'verdade,', 'hoje', 'em', 'dia', 'a', 'razão', 'e', 'o', 'amor', 'quase', 'não', 'andam', 'juntos.', 'É', 'pena', 'que', 'alguns', 'vizinhos', 'honestos', 'não', 'se', 'esforcem', 'para', 'torná-los', 'amigos.', 'Como', 'vê,', 'também', 'posso', 'ser', 'espirituoso', 'se', 'houver', 'ocasião.”', '(De', 'Sonho', 'de', 'uma', 'Noite', 'de', 'Verão)', '', '“A', 'vida', 'é', 'uma', 'simples', 'sombra', 'que', 'passa', '(...);', 'é', 'uma', 'história', 'contada', 'por', 'um', 'idiota,', 'cheia', 'de', 'ruído', 'e', 'de', 'furor', 'e', 'que', 'nada', 'significa”', '(De', 'Macbeth)']


In [10]:
#Contagem das palavras - Map
pairsRDD = wordsRDD.map(lambda x: (x, 1))

In [11]:
print(pairsRDD.collect())

[('', 1), ('“No', 1), ('entanto,', 1), ('para', 1), ('dizer', 1), ('a', 1), ('verdade,', 1), ('hoje', 1), ('em', 1), ('dia', 1), ('a', 1), ('razão', 1), ('e', 1), ('o', 1), ('amor', 1), ('quase', 1), ('não', 1), ('andam', 1), ('juntos.', 1), ('É', 1), ('pena', 1), ('que', 1), ('alguns', 1), ('vizinhos', 1), ('honestos', 1), ('não', 1), ('se', 1), ('esforcem', 1), ('para', 1), ('torná-los', 1), ('amigos.', 1), ('Como', 1), ('vê,', 1), ('também', 1), ('posso', 1), ('ser', 1), ('espirituoso', 1), ('se', 1), ('houver', 1), ('ocasião.”', 1), ('(De', 1), ('Sonho', 1), ('de', 1), ('uma', 1), ('Noite', 1), ('de', 1), ('Verão)', 1), ('', 1), ('“A', 1), ('vida', 1), ('é', 1), ('uma', 1), ('simples', 1), ('sombra', 1), ('que', 1), ('passa', 1), ('(...);', 1), ('é', 1), ('uma', 1), ('história', 1), ('contada', 1), ('por', 1), ('um', 1), ('idiota,', 1), ('cheia', 1), ('de', 1), ('ruído', 1), ('e', 1), ('de', 1), ('furor', 1), ('e', 1), ('que', 1), ('nada', 1), ('significa”', 1), ('(De', 1), ('Macbe

In [13]:
#Contagem final das palavras do arquivo - Reduce
frequenciesRDD = pairsRDD.reduceByKey(lambda x, y:x + y)
print(frequenciesRDD.collect())

[('', 2), ('para', 2), ('verdade,', 1), ('hoje', 1), ('em', 1), ('amor', 1), ('É', 1), ('que', 3), ('alguns', 1), ('torná-los', 1), ('amigos.', 1), ('Como', 1), ('vê,', 1), ('Sonho', 1), ('uma', 3), ('“A', 1), ('vida', 1), ('sombra', 1), ('passa', 1), ('(...);', 1), ('história', 1), ('contada', 1), ('um', 1), ('furor', 1), ('nada', 1), ('significa”', 1), ('Macbeth)', 1), ('“No', 1), ('entanto,', 1), ('dizer', 1), ('a', 2), ('dia', 1), ('razão', 1), ('e', 3), ('o', 1), ('quase', 1), ('não', 2), ('andam', 1), ('juntos.', 1), ('pena', 1), ('vizinhos', 1), ('honestos', 1), ('se', 2), ('esforcem', 1), ('também', 1), ('posso', 1), ('ser', 1), ('espirituoso', 1), ('houver', 1), ('ocasião.”', 1), ('(De', 2), ('de', 4), ('Noite', 1), ('Verão)', 1), ('é', 2), ('simples', 1), ('por', 1), ('idiota,', 1), ('cheia', 1), ('ruído', 1)]
